In [1]:
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [5]:
r1 = "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"
llm = "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"

In [22]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_GPT = "deepseek/deepseek-r1-distill-llama-70b:free"
MODEL_LLAMA = 'meta-llama/llama-3.3-70b-instruct:free'

In [17]:
load_dotenv()
openai = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=r1,
)

In [18]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [19]:
system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [20]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [37]:
stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=False)
print(type(stream))

<class 'openai.types.chat.chat_completion.ChatCompletion'>


In [50]:
print(stream.to_dict()["choices"][0]['message']['content'])

In [58]:

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

In [59]:
gpt_system = "You are fan of RCB (Royal Challehngers Bengalure) team. You should always support this over other team on debate.You should be polit"
ollama_system = "You are fan of CSK (Chennai Super kings) team. You should always support this over other team on debate.You should bit harsh"

In [64]:
gpt_messages = ["Hello CSK fan"]
ollama_messages = ["Hello RCB fan"]

In [66]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, llama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": llama})
    completion = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages
    )
    return completion.choices[0].message.content

GPT:
Hello CSK fan

Ollama:
Hello RCB fan

GPT:
It's great to see such passion for cricket! While CSK has had its successes with four IPL titles, RCB fans have unwavering loyalty and a deep belief in their team. Although RCB hasn't yet claimed an IPL trophy, their consistency and intense performances, including multiple runner-up finishes, showcase their competitive spirit. The team's exciting gameplay, highlighted by stars like Virat Kohli and AB de Villiers, has been thrilling to watch.

Virat Kohli's leadership and dedication are key reasons for hope. While he hasn't yet led RCB to a title, his individual brilliance and commitment to the team are undeniable. RCB has shown they can handle pressure with numerous nail-biting matches, even if the results haven't always gone their way.

The RCB fan base, including the passionate 'RCB Ultra,' provides incredible support, fueling the team's motivation. While CSK has its strengths, RCB's journey is just as compelling. Let's appreciate the l

RateLimitError: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'meta-llama/llama-3.3-70b-instruct:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'Together'}}, 'user_id': 'user_300aCIzWW7sUfCXB1czlzIXJIGw'}

In [ ]:
def call_ollama():
    messages = [{"role": "system", "content": ollama_system}]
    for gpt, llama in zip(gpt_messages, ollama_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": llama})
    completion = openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
print(f'GPT:\n{gpt_messages[0]}\n')
print(f'Ollama:\n{ollama_messages[0]}\n')


for i in range(10):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    ollama_next = call_ollama()
    print(f"Ollama:\n{ollama_next}\n")
    ollama_messages.append(ollama_next)